In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
## collab user
# !pip install neuron
# !pip install netpyne
# !git clone https://github.com/suny-downstate-medical-center/S1_netpyne.git
# import os
# os.chdir('S1_netpyne/info')

In [3]:
import matplotlib
import os
os.chdir('../sim')

In [4]:
# !nrnivmodl mod/

In [5]:
"""
cfg.py 

Simulation configuration for S1 model (using NetPyNE)
This file has sim configs as well as specification for parameterized values in netParams.py 

Contributors: salvadordura@gmail.com, fernandodasilvaborges@gmail.com
"""

from netpyne import specs, sim
import pickle, json
import numpy as np

%matplotlib inline

cfg = specs.SimConfig()  

#------------------------------------------------------------------------------
#
# SIMULATION CONFIGURATION
#
#------------------------------------------------------------------------------

#------------------------------------------------------------------------------
# Run parameters
#------------------------------------------------------------------------------
cfg.duration = 1.0*1e3 ## Duration of the sim, in ms  
cfg.dt = 0.025
cfg.seeds = {'conn': 4321, 'stim': 4321, 'loc': 4321} 
cfg.hParams = {'celsius': 34, 'v_init': -65}  
cfg.verbose = False
cfg.createNEURONObj = True
cfg.createPyStruct = True  
cfg.cvode_active = False
cfg.cvode_atol = 1e-6
cfg.cache_efficient = True
cfg.printRunTime = 0.1

cfg.includeParamsLabel = False
cfg.printPopAvgRates = True

cfg.checkErrors = False

cfg.poptypeNumber = 55
cfg.celltypeNumber = 207

In [6]:
#------------------------------------------------------------------------------  
# S1 Cells
# Load 55 Morphological Names and Cell pop numbers -> L1:6 L23:10 L4:12 L5:13 L6:14
# Load 207 Morpho-electrical Names used to import the cells from 'cell_data/' -> L1:14 L23:43 L4:46 L5:52 L6:52
# Create [Morphological,Electrical] = number of cell metype in the sub-pop

with open('../info/anatomy/S1-cells-distributions-Rat.txt') as mtype_file:
    mtype_content = mtype_file.read()       

cfg.popNumber = {}
cfg.cellNumber = {} 
cfg.popLabel = {} 
popParam = []
cellParam = []
cfg.meParamLabels = {} 
cfg.popLabelEl = {} 
cfg.cellLabel = {}

for line in mtype_content.split('\n')[:-1]:
    cellname, mtype, etype, n, m = line.split()
    metype = mtype + '_' + etype[0:3]
    cfg.cellNumber[metype] = int(n)
    cfg.popLabel[metype] = mtype
    cfg.popNumber[mtype] = int(m)
    cfg.cellLabel[metype] = cellname

    if mtype not in popParam:
        popParam.append(mtype)
        cfg.popLabelEl[mtype] = [] 
               
    cfg.popLabelEl[mtype].append(metype)
    
    cellParam.append(mtype + '_' + etype[0:3])
    

cfg.S1pops = popParam
cfg.S1cells = cellParam


cfg.popParamLabels = popParam
cfg.cellParamLabels = cellParam
#------------------------------------------------------------------------------  
# number=0
# for mtype in cfg.S1pops:
#     for cellEl in range(np.size(cfg.popLabelEl[mtype])):
#         number+=1
#         print(number,cfg.popLabelEl[mtype][cellEl], cfg.popLabel[cfg.popLabelEl[mtype][cellEl]], popLabelEl[cfg.popLabelEl[mtype][cellEl]])

In [7]:
#------------------------------------------------------------------------------
# Cells
#------------------------------------------------------------------------------

# TO DEBUG - import and simulate only the Cell soma (to study only the Net)
cfg.reducedtest = True    

# TO DEBUG - Create only 5 Cells for each MEtype in S1
cfg.oneCellperMEtypeS1 = False 

In [8]:
#------------------------------------------------------------------------------  
# TO DEBUG - Create only one Cell per MEtype in S1 cells
if cfg.oneCellperMEtypeS1:
	cfg.popNumber = {}
	cfg.cellNumber = {} 
	for mtype in cfg.S1pops:
		cfg.popNumber[mtype] = 0

	for line in mtype_content.split('\n')[:-1]:
		metype, mtype, etype, n, m = line.split()
		if int(n) < 5:
			cfg.cellNumber[metype] = int(n)
			cfg.popNumber[mtype] = cfg.popNumber[mtype] + int(n)
		else:
			cfg.cellNumber[metype] = 5
			cfg.popNumber[mtype] = cfg.popNumber[mtype] + 5


In [9]:
# testpop = ['L4_SS', 'L5_STPC', 'L5_UTPC', 'L23_LBC', 'L23_MC', 'L4_LBC', 'L4_MC', 'L5_LBC', 'L5_MC']
# Nsyn=513042, N=2684

In [10]:
subPopLabels = ['L5_SBC', 'L5_STPC']

for metype in cfg.cellParamLabels:
    if cfg.popLabel[metype] in subPopLabels:
        print(metype,cfg.cellNumber[metype],cfg.popLabel[metype],cfg.popNumber[cfg.popLabel[metype]])        

L5_SBC_bNA 9 L5_SBC 25
L5_SBC_cAC 9 L5_SBC 25
L5_SBC_dNA 7 L5_SBC 25
L5_STPC_cAD 302 L5_STPC 302


In [11]:
#------------------------------------------------------------------------------  
cfg.S1pops = subPopLabels
cfg.S1cells = []
for metype in cfg.cellParamLabels:
    if cfg.popLabel[metype] in subPopLabels:        
        cfg.S1cells.append(metype)
        
cfg.thalamicpops = []

cfg.popParamLabels = cfg.S1pops
cfg.cellParamLabels = cfg.S1cells

In [12]:
#--------------------------------------------------------------------------
# Recording 
#--------------------------------------------------------------------------
cfg.allpops = cfg.cellParamLabels
cfg.cellsrec = 2
if cfg.cellsrec == 0:  cfg.recordCells = cfg.allpops # record all cells
elif cfg.cellsrec == 1: cfg.recordCells = [(pop,0) for pop in cfg.allpops] # record one cell of each pop
elif cfg.cellsrec == 2: # record one cell of each cellMEtype
    cfg.recordCells = []
    for metype in cfg.cellParamLabels:
        if cfg.cellNumber[metype] < 5:
            for numberME in range(cfg.cellNumber[metype]):
                cfg.recordCells.append((metype,numberME))
        else:
            numberME = 0
            diference = cfg.cellNumber[metype] - 5.0*int(cfg.cellNumber[metype]/5.0)
            
            for number in range(5):            
                cfg.recordCells.append((metype,numberME))
                
                if number < diference:              
                    numberME+=int(np.ceil(cfg.cellNumber[metype]/5.0))  
                else:
                    numberME+=int(cfg.cellNumber[metype]/5.0)
                
        
cfg.recordTraces = {'V_soma': {'sec':'soma', 'loc':0.5, 'var':'v'}}  ## Dict with traces to record
cfg.recordStim = False
cfg.recordTime = False
cfg.recordStep = 0.1    

In [13]:
#------------------------------------------------------------------------------
# Saving
#------------------------------------------------------------------------------
cfg.simLabel = 'subNets_test0'
cfg.saveFolder = '../info/test/'+cfg.simLabel
# cfg.filename =                	## Set file output name
cfg.savePickle = False         	## Save pkl file
cfg.saveJson = True	           	## Save json file
cfg.saveDataInclude = ['simConfig','simData'] ## , 'netParams', 'simConfig', 
cfg.backupCfgFile = None 		##  
cfg.gatherOnlySimData = False	##  
cfg.saveCellSecs = False			
cfg.saveCellConns = True	

In [14]:
"""
netParams.py
"""
# Network parameters
netParams = specs.NetParams()  # object of class NetParams to store the network parameters

netParams.scale = 1.0 # Scale factor for number of cells
netParams.sizeX = 420.0 # x-dimension (horizontal length) size in um
netParams.sizeY = 2082.0 # y-dimension (vertical height or cortical depth) size in um
netParams.sizeZ = 420.0 # z-dimension (horizontal depth) size in um
netParams.shape = 'cylinder' # cylindrical (column-like) volume

# r = 210 um and hexagonal side length = 230.9 um

In [15]:
#------------------------------------------------------------------------------
# General network parameters
#------------------------------------------------------------------------------
# Layer	height (um)	height (norma)	from	to
# L1	165		    0.079		    0.000	0.079
# L2	149		    0.072		    0.079	0.151
# L3	353		    0.170		    0.151	0.320
# L4	190		    0.091		    0.320	0.412
# L5	525		    0.252		    0.412	0.664
# L6	700		    0.336		    0.664	1.000
# L23	502		    0.241		    0.079	0.320
# All	2082	    1.000	


cellModels = ['HH_full']
Epops = ['L23_PC', 'L4_PC', 'L4_SS', 'L4_SP', 
             'L5_TTPC1', 'L5_TTPC2', 'L5_STPC', 'L5_UTPC',
             'L6_TPC_L1', 'L6_TPC_L4', 'L6_BPC', 'L6_IPC', 'L6_UTPC']
Ipops = []
for popName in cfg.S1pops:
    if popName not in Epops:
        Ipops.append(popName)

Epops = []
for popName in cfg.S1pops:
    if popName not in Ipops:
        Epops.append(popName)   

layer = {'1':[0.0, 0.079], '2': [0.079,0.151], '3': [0.151,0.320], '23': [0.079,0.320], '4':[0.320,0.412], '5': [0.412,0.664], '6': [0.664,1.0], 
'longS1': [2.2,2.3], 'longS2': [2.3,2.4]}  # normalized layer boundaries


In [16]:
#------------------------------------------------------------------------------
# General connectivity parameters
#------------------------------------------------------------------------------
netParams.defaultThreshold = -10.0 # spike threshold, 10 mV is NetCon default, lower it for all cells
netParams.defaultDelay = 0.1 # default conn delay (ms)(M1: 2.0 ms)
netParams.propVelocity = 300.0 #  300 μm/ms (Stuart et al., 1997)(M1: 500.0um/ms)
netParams.scaleConnWeight = 0.001 # weight conversion factor (from nS to uS)
netParams.scaleConnWeightNetStims = 0.001  # weight conversion factor (from nS to uS)

In [17]:
#------------------------------------------------------------------------------
# Population parameters
#------------------------------------------------------------------------------
# S1
cfg.scaleDensity = 1.0

# for popName in cfg.S1pops:
# 	layernumber = popName[1:2]
# 	if layernumber == '2':
# 		netParams.popParams[popName] = {'cellType': popName, 'cellModel': 'HH_full', 'ynormRange': layer['23'], 
#                                         'numCells': int(np.ceil(cfg.scaleDensity*cfg.popNumber[popName])), 'diversity': True}
# 	else:
# 		netParams.popParams[popName] = {'cellType': popName, 'cellModel': 'HH_full', 'ynormRange': layer[layernumber], 
#                                         'numCells': int(np.ceil(cfg.scaleDensity*cfg.popNumber[popName])), 'diversity': True}


for metype in cfg.cellParamLabels:
    if cfg.popLabel[metype] in subPopLabels:
        print(metype,cfg.cellNumber[metype],cfg.popLabel[metype],cfg.popNumber[cfg.popLabel[metype]])     
        netParams.popParams[metype] = {'cellType': metype, 'cellModel': 'HH_full', 'ynormRange': layer['5'], 
                                        'numCells': int(np.ceil(cfg.scaleDensity*cfg.cellNumber[metype])), 'diversity': True}

L5_SBC_bNA 9 L5_SBC 25
L5_SBC_cAC 9 L5_SBC 25
L5_SBC_dNA 7 L5_SBC 25
L5_STPC_cAD 302 L5_STPC 302


In [18]:
## S1 cell property rules
cellnumber = 0    
for cellName in cfg.S1cells:
    
    if cfg.cellNumber[cellName] < 5:
        morphoNumbers = cfg.cellNumber[cellName]
    else:
        morphoNumbers = 5
    
    cellFraction = 1.0/morphoNumbers
    
    for morphoNumber in range(morphoNumbers):
        cellMe = cfg.cellLabel[cellName] + '_' + str(morphoNumber+1)
        ## Load cell rules previously saved using netpyne format
        netParams.loadCellParamsRule(label = cellMe, fileName = 'cell_data/' + cellMe + '/' + cellMe + '_cellParams.pkl')    
        netParams.renameCellParamsSec(label = cellMe, oldSec = 'soma_0', newSec = 'soma')

        cellRule = {'conds': {'cellType': cellName}, 'diversityFraction': cellFraction, 'secs': {}}  # cell rule dict
        cellRule['secs'] = netParams.cellParams[cellMe]['secs']     
        cellRule['conds'] = netParams.cellParams[cellMe]['conds']    
        cellRule['conds']['cellType'] = cellName
        cellRule['globals'] = netParams.cellParams[cellMe]['globals']       
        cellRule['secLists'] = netParams.cellParams[cellMe]['secLists']                 
        cellRule['secLists']['all'][0] = 'soma' # replace 'soma_0'
        cellRule['secLists']['somatic'][0]  = 'soma' # replace 'soma_0'
                              
        cellRule['secLists']['spiny'] = {}
        cellRule['secLists']['spinyEE'] = {}

        nonSpiny = ['axon_0', 'axon_1']
        cellRule['secLists']['spiny'] = [sec for sec in cellRule['secLists']['all'] if sec not in nonSpiny]
        nonSpinyEE = ['axon_0', 'axon_1', 'soma']
        cellRule['secLists']['spinyEE'] = [sec for sec in cellRule['secLists']['all'] if sec not in nonSpinyEE]

        #-----------------------------------------------------------------------------------#
        if cfg.reducedtest:
            cellRule['secs'] = {}
            cellRule['secs']['soma'] = netParams.cellParams[cellMe]['secs']['soma']
            cellRule['secLists']['spiny'] = ['soma']
            cellRule['secLists']['spinyEE'] = ['soma']
            cellRule['secLists']['all'] = ['soma']
            cellRule['secLists']['basal'] = ['soma']    
        #-----------------------------------------------------------------------------------#
        netParams.cellParams[cellMe] = cellRule   # add dict to list of cell params   
        cellnumber=cellnumber+1

In [19]:
# ## S1 cell property rules
# cellnumber = 0    
# for cellName in cfg.S1cells:
    
#     if cfg.cellNumber[cellName] < 5:
#         morphoNumbers = cfg.cellNumber[cellName]
#     else:
#         morphoNumbers = 5
    
#     popName = cfg.popLabel[cellName]
#     cellFraction = 1.0*cfg.cellNumber[cellName]/(morphoNumbers*cfg.popNumber[popName])
    
#     for morphoNumber in range(morphoNumbers):
#         cellMe = cellName + '_' + str(morphoNumber+1)
#         ## Load cell rules previously saved using netpyne format
#         netParams.loadCellParamsRule(label = cellMe, fileName = 'cell_data/' + cellMe + '/' + cellMe + '_cellParams.pkl')    
#         netParams.renameCellParamsSec(label = cellMe, oldSec = 'soma_0', newSec = 'soma')

#         cellRule = {'conds': {'cellType': popName}, 'diversityFraction': cellFraction, 'secs': {}}  # cell rule dict
#         cellRule['secs'] = netParams.cellParams[cellMe]['secs']     
#         cellRule['conds'] = netParams.cellParams[cellMe]['conds']    
#         cellRule['conds']['cellType'] = popName
#         cellRule['globals'] = netParams.cellParams[cellMe]['globals']       
#         cellRule['secLists'] = netParams.cellParams[cellMe]['secLists']                 
#         cellRule['secLists']['all'][0] = 'soma' # replace 'soma_0'
#         cellRule['secLists']['somatic'][0]  = 'soma' # replace 'soma_0'
                              
#         cellRule['secLists']['spiny'] = {}
#         cellRule['secLists']['spinyEE'] = {}

#         nonSpiny = ['axon_0', 'axon_1']
#         cellRule['secLists']['spiny'] = [sec for sec in cellRule['secLists']['all'] if sec not in nonSpiny]
#         nonSpinyEE = ['axon_0', 'axon_1', 'soma']
#         cellRule['secLists']['spinyEE'] = [sec for sec in cellRule['secLists']['all'] if sec not in nonSpinyEE]

#         #-----------------------------------------------------------------------------------#
#         if cfg.reducedtest:
#             cellRule['secs'] = {}
#             cellRule['secs']['soma'] = netParams.cellParams[cellMe]['secs']['soma']
#             cellRule['secLists']['spiny'] = ['soma']
#             cellRule['secLists']['spinyEE'] = ['soma']
#             cellRule['secLists']['all'] = ['soma']
#             cellRule['secLists']['basal'] = ['soma']    
#         #-----------------------------------------------------------------------------------#
#         netParams.cellParams[cellMe] = cellRule   # add dict to list of cell params   
#         cellnumber=cellnumber+1

In [20]:
#------------------------------------------------------------------------------
# Synaptic mechanism parameters  - mods from M1 detailed
#------------------------------------------------------------------------------
## S1
netParams.synMechParams['AMPA'] = {'mod':'MyExp2SynBB', 'tau1': 0.2, 'tau2': 1.74, 'e': 0}
netParams.synMechParams['NMDA'] = {'mod': 'MyExp2SynNMDABB', 'tau1NMDA': 0.29, 'tau2NMDA': 43, 'e': 0}
netParams.synMechParams['GABAA6'] = {'mod':'MyExp2SynBB', 'tau1': 0.2, 'tau2': 6.44, 'e': -80}
netParams.synMechParams['GABAA'] = {'mod':'MyExp2SynBB', 'tau1': 0.2, 'tau2': 8.3, 'e': -80}
netParams.synMechParams['GABAA10'] = {'mod':'MyExp2SynBB', 'tau1': 0.2, 'tau2': 10.4, 'e': -80}
netParams.synMechParams['GABAB'] = {'mod':'MyExp2SynBB', 'tau1': 3.5, 'tau2': 260.9, 'e': -93} 

ESynMech = ['AMPA', 'NMDA']
ISynMech = ['GABAA', 'GABAB']
ISynMech6 = ['GABAA6', 'GABAB']
ISynMech10 = ['GABAA10', 'GABAB']

In [21]:
#------------------------------------------------------------------------------
# load data from S1 conn pre-processing file 
#------------------------------------------------------------------------------
with open('conn/conn.pkl', 'rb') as fileObj: connData = pickle.load(fileObj)

lmat = connData['lmat']
a0mat = connData['a0mat']
d0 = connData['d0']
dfinal = connData['dfinal']
pmat = {}
pmat[12.5] = connData['pmat12um']
pmat[25] = connData['pmat25um']
pmat[50] = connData['pmat50um']
pmat[75] = connData['pmat75um']
pmat[100] = connData['pmat100um']
pmat[125] = connData['pmat125um']
pmat[150] = connData['pmat150um']
pmat[175] = connData['pmat175um']
pmat[200] = connData['pmat200um'] #max value for d0=200

synperconnNumber = connData['synperconnNumber']
connNumber = connData['connNumber']
decay = connData['decay']
gsyn = connData['gsyn']
use = connData['use']


ConnTypesNumber = connData['ConnTypesNumber'] 
ConnTypes = connData['ConnTypes']   
connIEtype = connData['connIEtype']  
connEItype = connData['connEItype']
parameters_syn = connData['parameters_syn']

In [30]:
import pandas as pd

physColumnNames = []
syntypes = []
for name,syntype in parameters_syn:    
    if name not in physColumnNames:
        physColumnNames.append(name) 
    if syntype not in syntypes:
        syntypes.append(syntype)
        
#     print(name,syntype,parameters_syn[name,syntype])
dfS6 = pd.DataFrame(index=syntypes, columns=physColumnNames)
for syntype in syntypes:
    for name in physColumnNames:    
        dfS6.loc[syntype][name] = parameters_syn[name,syntype]
dfS6

,ConnType,gsyn,gsynStd,decay,decayStd,use,useStd,dep,depStd,fac,facStd,s-types,type
0,0,0.83,0.2,10.4,6.1,0.16,0.1,45.0,21.0,376.0,253.0,I1,IE
1,1,0.83,0.55,8.3,2.2,0.25,0.13,706.0,405.0,21.0,9.0,I2,II
3,2,0.91,0.61,10.4,6.1,0.16,0.1,45.0,21.0,376.0,253.0,I1,IE
4,3,0.91,0.61,8.3,2.2,0.25,0.13,706.0,405.0,21.0,9.0,I2,IE
5,4,0.91,0.61,6.44,1.7,0.32,0.14,144.0,80.0,62.0,31.0,I3,IE
8,5,0.75,0.32,8.3,2.2,0.25,0.13,706.0,405.0,21.0,9.0,I2,IE
9,6,0.75,0.32,8.3,2.2,0.3,0.08,1250.0,520.0,2.0,4.0,I2,IE
10,7,0.91,0.61,8.3,2.2,0.14,0.05,875.0,285.0,22.0,5.0,I2,IE
11,8,0.83,0.2,10.4,6.1,0.25,0.13,706.0,405.0,21.0,9.0,I3,IE
12,9,0.91,0.61,8.3,2.2,0.25,0.13,706.0,405.0,21.0,9.0,I2,IE


In [ ]:
#------------------------------------------------------------------------------
# Synaptic mechanism parameters  - mods from S1 BBP - deterministic version
#------------------------------------------------------------------------------
synmechparams = {}
for pre in Epops+Ipops:
    for post in Epops+Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:  
          if pre in Epops:
            mod = 'DetAMPANMDA'
            tau = 'tau_d_AMPA'
          else:
            mod = 'DetGABAAB'
            tau = 'tau_d_GABAA'

            conn = connType[ConnTypes[pre][post][0]]

            synmechparams[proj] = {'mod': mod,
                                     tau: dfS6['decay'][conn],
                                     #'tau_d_NMDA': 61.0, 
                                     'Use': dfS6['use'][conn],
                                     'Dep': dfS6['dep'][conn], 
                                     'Fac': dfS6['fac'][conn], 
                                     'gmax': dfS6['gsyn'][conn]}

In [33]:
#------------------------------------------------------------------------------
# S1 Local connectivity parameters 
#------------------------------------------------------------------------------
cfg.addConn = True

cfg.synWeightFractionEE = [1.0, 1.0] # E -> E AMPA to NMDA ratio
cfg.synWeightFractionEI = [1.0, 1.0] # E -> I AMPA to NMDA ratio
cfg.synWeightFractionII = [1.0, 1.0]  # I -> I GABAA to GABAB ratio
cfg.synWeightFractionIE = [1.0, 1.0]  # I -> E GABAA to GABAB ratio
cfg.EEGain = 1.0
cfg.EIGain = 1.0
cfg.IIGain = 1.0
cfg.IEGain = 1.0

if cfg.addConn:    
#------------------------------------------------------------------------------    
# I -> I
#------------------------------------------------------------------------------
    for pre in Ipops:
        for post in Ipops:
            if float(connNumber[pre][post]) > 0:        

                if int(float(d0[pre][post])) < 25:    #d0==12.5 -> single exponential fit
                    linear = 0
                    angular = 0
                    prob = '%s * exp(-dist_2D/%s)*(dist_2D<%s)' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post])                     
                elif int(float(d0[pre][post])) == 25:    #d0==25 -> exponential fit when dist_2D>25, else prob[0um:25um] = pmat[12.5]
                    linear = float(pmat[12.5][pre][post])
                    angular = 0
                    prob = '%s * exp(-dist_2D/%s)*(dist_2D<%s) if dist_2D > %s else %f * dist_2D + %f' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post],d0[pre][post],angular,linear)
                else:    #d0>25 -> exponential fit when dist_2D>d0, else prob[0um:d0] = linear interpolation [25:d0]
                    d01 = int(float(d0[pre][post]))
                    y1 = float(pmat[25][pre][post])
                    y2 = float(pmat[d01][pre][post])
                    x1 = 25
                    x2 = d01                   
                    angular = (y2 - y1)/(x2 - x1)
                    linear = y2 - x2*angular
                    prob = '%s * exp(-dist_2D/%s)*(dist_2D<%s) if dist_2D > %s else %f * dist_2D + %f' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post],d0[pre][post],angular,linear)
                
                if decay[pre][post] > 10.0:
                    synMechType =  ISynMech10
                elif decay[pre][post] < 7.0:
                    synMechType =  ISynMech6
                else:
                    synMechType =  ISynMech

                for npre,cellpre in enumerate(cfg.popLabelEl[pre]):
                    for npost,cellpost in enumerate(cfg.popLabelEl[post]):
                        print(cfg.popLabelEl[pre][npre],cfg.popLabelEl[post][npost])    
                    
                        netParams.connParams['II_'+cellpre+'_'+cellpost] = { 
                            'preConds': {'pop': cellpre}, 
                            'postConds': {'pop': cellpost},
                            'synMech': synMechType,
                            'probability': prob,
                            'weight': gsyn[pre][post] * cfg.IIGain, 
                            'synMechWeightFactor': cfg.synWeightFractionII,
                            'delay': 'defaultDelay+dist_3D/propVelocity',
                            'synsPerConn': int(synperconnNumber[pre][post]+0.5),
                            'sec': 'spiny'}       
#------------------------------------------------------------------------------
## I -> E
#------------------------------------------------------------------------------
    for pre in Ipops:
        for post in Epops:
            if float(connNumber[pre][post]) > 0:        

                if int(float(d0[pre][post])) < 25:    #d0==12.5 -> single exponential fit
                    linear = 0
                    angular = 0
                    prob = '%s*exp(-dist_2D/%s)*(dist_2D<%s)' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post])                     
                elif int(float(d0[pre][post])) == 25:    #d0==25 -> exponential fit when dist_2D>25, else prob[0um:25um] = pmat[12.5]
                    linear = float(pmat[12.5][pre][post])
                    angular = 0
                    prob = '%s*exp(-dist_2D/%s)*(dist_2D<%s) if dist_2D > %s else %f*dist_2D+%f' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post],d0[pre][post],angular,linear)
                else:    #d0>25 -> exponential fit when dist_2D>d0, else prob[0um:d0] = linear interpolation [25:d0]
                    d01 = int(float(d0[pre][post]))
                    y1 = float(pmat[25][pre][post])
                    y2 = float(pmat[d01][pre][post])
                    x1 = 25
                    x2 = d01                   
                    angular = (y2 - y1)/(x2 - x1)
                    linear = y2 - x2*angular
                    prob = '%s*exp(-dist_2D/%s)*(dist_2D<%s) if dist_2D > %s else %f * dist_2D + %f' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post],d0[pre][post],angular,linear)

                if decay[pre][post] > 10.0:
                    synMechType =  ISynMech10
                elif decay[pre][post] < 7.0:
                    synMechType =  ISynMech6
                else:
                    synMechType =  ISynMech
                    
                print(pre,post)
                for npre,cellpre in enumerate(cfg.popLabelEl[pre]):
                    for npost,cellpost in enumerate(cfg.popLabelEl[post]):
                        
                        # ME conn diversity IE
                        if 'SBC' in pre or 'LBC' in pre or 'NBC' in pre: 
                            
                                premtype = pre[-3:]
                                preetype = cellpre[-3:]
                                connID = connIEtype[premtype][preetype]
                                
                                print('%s:%s (ID_BBP=%d or ID_new=%d)  \n gsyn=%4.4f decay=%4.4f use=%4.4f dep=%4.4f fac=%4.4f (%s , %s)' % (cellpre,cellpost,connID,parameters_syn['ConnType',connID]
                                                                                           ,parameters_syn['gsyn',connID], 
                                                                               parameters_syn['decay',connID],
                                    parameters_syn['use',connID],parameters_syn['dep',connID],parameters_syn['fac',connID],
                                    parameters_syn['s-types',connID],parameters_syn['type',connID]))                               
                                    
                                
                                netParams.connParams['IE_'+cellpre+'_'+cellpost] = { 
                                    'preConds': {'pop': cellpre}, 
                                    'postConds': {'pop': cellpost},
                                    'synMech': synMechType,
                                    'probability': prob,
                                    'weight': gsyn[pre][post] * cfg.IEGain, 
                                    'synMechWeightFactor': cfg.synWeightFractionIE,
                                    'delay': 'defaultDelay+dist_3D/propVelocity',
                                    'synsPerConn': int(synperconnNumber[pre][post]+0.5),
                                    'sec': 'spiny'}  
                                
                        else: 
                                print(cfg.popLabelEl[pre][npre],cfg.popLabelEl[post][npost])    
                                netParams.connParams['IE_'+cellpre+'_'+cellpost] = { 
                                    'preConds': {'pop': cellpre}, 
                                    'postConds': {'pop': cellpost},
                                    'synMech': synMechType,
                                    'probability': prob,
                                    'weight': gsyn[pre][post] * cfg.IEGain, 
                                    'synMechWeightFactor': cfg.synWeightFractionIE,
                                    'delay': 'defaultDelay+dist_3D/propVelocity',
                                    'synsPerConn': int(synperconnNumber[pre][post]+0.5),
                                    'sec': 'spiny'}  
                                
                                
                                
        # #------------------------------------------------------------------------------   
# ## E -> E
#     for pre in Epops:
#         for post in Epops:
#             if float(connNumber[pre][post]) > 0:        

#                 if int(float(d0[pre][post])) < 25:    #d0==12.5 -> single exponential fit
#                     linear = 0
#                     angular = 0
#                     prob = '%s * exp(-dist_2D/%s)*(dist_2D<%s)' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post])                     
#                 elif int(float(d0[pre][post])) == 25:    #d0==25 -> exponential fit when dist_2D>25, else prob[0um:25um] = pmat[12.5]
#                     linear = float(pmat[12.5][pre][post])
#                     angular = 0
#                     prob = '%s * exp(-dist_2D/%s)*(dist_2D<%s) if dist_2D > %s else %f * dist_2D + %f' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post],d0[pre][post],angular,linear)
#                 else:    #d0>25 -> exponential fit when dist_2D>d0, else prob[0um:d0] = linear interpolation [25:d0]
#                     d01 = int(float(d0[pre][post]))
#                     y1 = float(pmat[25][pre][post])
#                     y2 = float(pmat[d01][pre][post])
#                     x1 = 25
#                     x2 = d01                   
#                     angular = (y2 - y1)/(x2 - x1)
#                     linear = y2 - x2*angular
#                     prob = '%s * exp(-dist_2D/%s)*(dist_2D<%s) if dist_2D > %s else %f * dist_2D + %f' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post],d0[pre][post],angular,linear)

#                 netParams.connParams['EE_'+pre+'_'+post] = { 
#                     'preConds': {'pop': pre}, 
#                     'postConds': {'pop': post},
#                     'synMech': ESynMech,
#                     'probability': prob, 
#                     'weight': gsyn[pre][post] * cfg.EEGain, 
#                     'synMechWeightFactor': cfg.synWeightFractionEE,
#                     'delay': 'defaultDelay+dist_3D/propVelocity',
#                     'synsPerConn': int(synperconnNumber[pre][post]+0.5),
#                     'sec': 'spinyEE'}    
# # ## E -> I
#     for pre in Epops:
#         for post in Ipops:
#             if float(connNumber[pre][post]) > 0:        

#                 if int(float(d0[pre][post])) < 25:    #d0==12.5 -> single exponential fit
#                     linear = 0
#                     angular = 0
#                     prob = '%s * exp(-dist_2D/%s)*(dist_2D<%s)' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post])                     
#                 elif int(float(d0[pre][post])) == 25:    #d0==25 -> exponential fit when dist_2D>25, else prob[0um:25um] = pmat[12.5]
#                     linear = float(pmat[12.5][pre][post])
#                     angular = 0
#                     prob = '%s * exp(-dist_2D/%s)*(dist_2D<%s) if dist_2D > %s else %f * dist_2D + %f' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post],d0[pre][post],angular,linear)
#                 else:    #d0>25 -> exponential fit when dist_2D>d0, else prob[0um:d0] = linear interpolation [25:d0]
#                     d01 = int(float(d0[pre][post]))
#                     y1 = float(pmat[25][pre][post])
#                     y2 = float(pmat[d01][pre][post])
#                     x1 = 25
#                     x2 = d01                   
#                     angular = (y2 - y1)/(x2 - x1)
#                     linear = y2 - x2*angular
#                     prob = '%s * exp(-dist_2D/%s)*(dist_2D<%s) if dist_2D > %s else %f * dist_2D + %f' % (a0mat[pre][post],lmat[pre][post],dfinal[pre][post],d0[pre][post],angular,linear)

#                 netParams.connParams['EI_'+pre+'_'+post] = { 
#                     'preConds': {'pop': pre}, 
#                     'postConds': {'pop': post},
#                     'synMech': ESynMech,
#                     'probability': prob, 
#                     'weight': gsyn[pre][post] * cfg.EIGain, 
#                     'synMechWeightFactor': cfg.synWeightFractionEI,
#                     'delay': 'defaultDelay+dist_3D/propVelocity',
#                     'synsPerConn': int(synperconnNumber[pre][post]+0.5),
#                     'sec': 'spiny'}    
                   

L5_SBC L5_STPC
L5_SBC_bNA:L5_STPC_cAD (ID_BBP=5 or ID_new=4)  
 gsyn=0.9100 decay=6.4400 use=0.3200 dep=144.0000 fac=62.0000 (I3 , IE)
L5_SBC_cAC:L5_STPC_cAD (ID_BBP=3 or ID_new=2)  
 gsyn=0.9100 decay=10.4000 use=0.1600 dep=45.0000 fac=376.0000 (I1 , IE)
L5_SBC_dNA:L5_STPC_cAD (ID_BBP=4 or ID_new=3)  
 gsyn=0.9100 decay=8.3000 use=0.2500 dep=706.0000 fac=21.0000 (I2 , IE)


In [ ]:
for connpath in netParams.connParams.keys():
    print(connpath)
    print(netParams.connParams[connpath])
    print()

In [ ]:
# #------------------------------------------------------------------------------
# # Spontaneous synapses + background - data from Rat
# #------------------------------------------------------------------------------
# cfg.addStimSynS1 = True
# cfg.rateStimE = 6.0
# cfg.rateStimI = 9.0

# #------------------------------------------------------------------------------
# # NetStim inputs to simulate Spontaneous synapses + background in S1 neurons - data from Rat
# #------------------------------------------------------------------------------
# SourcesNumber = 5 # for each post Mtype - sec distribution
# synperNeuronStimI = connData['synperNeuronStimI']
# synperNeuronStimE = connData['synperNeuronStimE']
# GsynStimI = connData['GsynStimI']
# GsynStimE = connData['GsynStimE']
   
# if cfg.addStimSynS1:      
#     for post in Ipops + Epops:

#         synperNeuron = synperNeuronStimI[post]
#         ratespontaneous = cfg.rateStimI
#         for qSnum in range(SourcesNumber):
#             ratesdifferentiation = (0.8 + 0.4*qSnum/(SourcesNumber-1)) * (synperNeuron*ratespontaneous)/SourcesNumber
#             netParams.stimSourceParams['StimSynS1_S_all_INH->' + post + '_' + str(qSnum)] = {'type': 'NetStim', 'rate': ratesdifferentiation, 'noise': 1.0}

#         synperNeuron = synperNeuronStimE[post]
#         ratespontaneous = cfg.rateStimE
#         for qSnum in range(SourcesNumber):
#             ratesdifferentiation = (0.8 + 0.4*qSnum/(SourcesNumber-1)) * (synperNeuron*ratespontaneous)/SourcesNumber
#             netParams.stimSourceParams['StimSynS1_S_all_EXC->' + post + '_' + str(qSnum)] = {'type': 'NetStim', 'rate': ratesdifferentiation, 'noise': 1.0}
            
#     #------------------------------------------------------------------------------
#     for post in Epops:
#         for qSnum in range(SourcesNumber):
#             netParams.stimTargetParams['StimSynS1_T_all_EXC->' + post + '_' + str(qSnum)] = {
#                 'source': 'StimSynS1_S_all_EXC->' + post + '_' + str(qSnum), 
#                 'conds': {'cellType': post}, 
#                 'ynorm':[0,1], 
#                 'sec': 'spinyEE', 
#                 'loc': 0.5, 
#                 'synMechWeightFactor': [1.0],
#                 'weight': GsynStimE[post],
#                 'delay': 0.1, 
#                 'synMech': 'AMPA'}

#     for post in Ipops:
#         for qSnum in range(SourcesNumber):
#             netParams.stimTargetParams['StimSynS1_T_all_EXC->' + post + '_' + str(qSnum)] = {
#                 'source': 'StimSynS1_S_all_EXC->' + post + '_' + str(qSnum), 
#                 'conds': {'cellType': post}, 
#                 'ynorm':[0,1], 
#                 'sec': 'spiny', 
#                 'loc': 0.5, 
#                 'synMechWeightFactor': [1.0],
#                 'weight': GsynStimE[post],
#                 'delay': 0.1, 
#                 'synMech': 'AMPA'}

#     for post in Epops+Ipops:
#         for qSnum in range(SourcesNumber):
#             netParams.stimTargetParams['StimSynS1_T_all_INH->' + post + '_' + str(qSnum)] = {
#                 'source': 'StimSynS1_S_all_INH->' + post + '_' + str(qSnum), 
#                 'conds': {'cellType': post}, 
#                 'ynorm':[0,1], 
#                 'sec': 'spiny', 
#                 'loc': 0.5, 
#                 'synMechWeightFactor': [1.0],
#                 'weight': GsynStimI[post],
#                 'delay': 0.1, 
#                 'synMech': 'GABAA'}

In [ ]:
# for stimpath in netParams.stimSourceParams.keys():
#     print(stimpath)
#     print(netParams.stimSourceParams[stimpath])
#     print()    

In [ ]:
# for stimpath in netParams.stimTargetParams.keys():
#     print(stimpath)
#     print(netParams.stimTargetParams[stimpath])
#     print()    

## Run small Nets in a sigle job

In [ ]:
sim.initialize(
    simConfig = cfg, 	
    netParams = netParams)  				# create network object and set cfg and net params
sim.net.createPops()               			# instantiate network populations
sim.net.createCells()              			# instantiate network cells based on defined populations
sim.net.connectCells()            			# create connections between cells based on params
sim.net.addStims() 							# add network stimulation
sim.setupRecording()              			# setup variables to record for each cell (spikes, V traces, etc)
sim.runSim()                      			# run parallel Neuron simulation  
sim.gatherData()                  			# gather spiking data and cell info from each node
sim.saveData()                    			# save params, cell info and sim output to file (pickle,mat,txt,etc)#
sim.analysis.plotData()  

## Net infos

In [ ]:
# features = ['numConns','convergence'] includePre=subPopLabels, includePost = subPopLabels,
test = sim.analysis.plotConn( feature='convergence', figSize=(6, 6), fontSize=8, saveData=None, 
                               saveFig=True, showFig=False);

In [ ]:
test[1]['connMatrix'] # total of synapses [AMPA, NMDA] or [GABAA, GABAB]

In [ ]:
sim.analysis.plot2Dnet(view='xy', showConns=True, 
                               popColors=None, tagsFile=None, figSize=(12, 12), fontSize=8, saveData=None, 
                               saveFig=True, showFig=True, lineWidth=0.1);
# sim.analysis.plot2Dnet(include=subPopLabels,view='xy', showConns=False, 
#                                popColors=None, tagsFile=None, figSize=(8, 8), fontSize=12, saveData=None, 
#                                saveFig=None, showFig=True, lineWidth=0.1);
# sim.analysis.plot2Dnet(include=subPopLabels,view='xz', showConns=False, 
#                                popColors=None, tagsFile=None, figSize=(8, 8), fontSize=12, saveData=None, 
#                                saveFig=None, showFig=True, lineWidth=0.1);

## Raster and traces

In [ ]:
sim.analysis.plotRaster(include=subPopLabels, timeRange=[0,cfg.duration], orderBy='gid', orderInverse=True, labels='legend', popRates=True, lw=5, marker='.', markerSize=24, figSize=(20, 10), fontSize=20, dpi=300, saveFig=True, showFig=False);

In [ ]:
cellNumber[metype]for metype in cfg.cellParamLabels:
    if cfg.popLabel[metype] in subPopLabels:
        print(metype,cfg.cellNumber[metype],cfg.popLabel[metype],cfg.popNumber[cfg.popLabel[metype]])     

In [ ]:
traces = [(mtype, [i for i in range(0,cfg.popNumber[mtype],int(cfg.popNumber[mtype]/4.5))]) for mtype in subPopLabels]
traces

In [ ]:
sim.analysis.plotTraces(include=traces, timeRange=[0,cfg.duration], overlay=False, oneFigPer='trace', ylim=[-90,50], subtitles= False, axis=False, scaleBarLoc=1, figSize=(18, 8), fontSize=12, saveFig=True);

In [ ]:
subPopLabels

In [ ]:
for pops in subPopLabels:
    print(pops)
    sim.analysis.plotTraces(include=[pops], timeRange=[0,cfg.duration], overlay=True, oneFigPer='trace', subtitles=False, legend=False, ylim=[-110,50], axis=False, scaleBarLoc=3, figSize=(18, 4), fontSize=15, saveFig='../info/test/'+cfg.simLabel+'/'+cfg.simLabel +'_'+pops+ '_Vt.png');

In [ ]:
netParams.cellParams.keys()